In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("diamonds.csv")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [3]:
# 알아서 바로 기존 컬럼들을 삭제해준다. 다른변환이 없다면 처음에 바로 사용하기 편함
df_dummy = pd.get_dummies(df, drop_first=True)
df_dummy.head()

,carat,depth,table,price,x,y,z,cut_Good,cut_Ideal,cut_Premium,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,1,...,0,1,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,1,0,0,...,0,0,1,0,0,1,0,0,0,0


In [4]:
df["rolling"] = df["price"].rolling(4).mean()

In [5]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,rolling
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,NaN
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,NaN
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,NaN
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,328.25
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,330.50


In [6]:
from scipy.stats import levene
df = pd.read_excel("iris_xlsx.xlsx")
df.head(3)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [7]:
stat, p = levene(df["Sepal.Length"], df["Sepal.Width"])

In [8]:
print(stat)
print(p)

62.89202775428331
4.4284985933065796e-14


In [12]:
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
ttest_1samp(df["Sepal.Length"], popmean=4)

Ttest_1sampResult(statistic=27.263680640799215, pvalue=8.764592435410748e-60)

In [13]:
ttest_rel(df["Sepal.Length"], df["Sepal.Width"])

Ttest_relResult(statistic=34.81519257082818, pvalue=1.8495537347987982e-73)

In [14]:
ttest_ind(df["Sepal.Length"], df["Sepal.Width"], equal_var=True)

Ttest_indResult(statistic=36.463283934449116, pvalue=7.027919392009369e-112)

In [16]:
#일원 분산분석
# 종속변수 1개, 독립변수 1개인 경우 각 집단간 평균 비교를 위해 사용하는 방법(3개 이상의 집단)
# 이런 컬럼들의 값을 하나의 집단으로 간주
# 귀무가설 : 전체 집단간 평균 차이는 같다
# 대립가설 : 전체 집단간 평균 차이는 같지않다.

from scipy.stats import f_oneway
f_oneway(df["Sepal.Length"], df["Sepal.Width"], df["Petal.Length"])

F_onewayResult(statistic=236.75163298404746, pvalue=7.657188374182084e-71)

In [ ]:
# 위 아래 동일하나 아래가 좀더 고급...

In [18]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

df = pd.read_csv("diamonds.csv")
formula = "price ~ C(cut)"
lm = ols(formula, df).fit()
result = anova_lm(lm)
print(result)

               df        sum_sq       mean_sq           F         PR(>F)
C(cut)        4.0  1.104175e+10  2.760436e+09  175.688717  8.428307e-150
Residual  53935.0  8.474314e+11  1.571209e+07         NaN            NaN


In [20]:
result.sum_sq.sum()

858473135517.3965

In [23]:
# 사후검정 - Tukey's HSD test
# 분산분석을 통해 세 그룹간의 평균 차이가 있음을 알수 있지만 어느 것이 차이가 있는지는 알수없다
# 구체적으로 어느 그룹이 차이 있는지 검정하는 것이 사후검정

from statsmodels.stats.multicomp import MultiComparison

mcomp = MultiComparison(df["price"], df["cut"])
mcomp.tukeyhsd().summary()

# reject colums은 귀무가설 기각여부
# 귀무가설을 기각하지 않는 조합케이스를 구하라
# Fair	Premium	
# Good	Very Good

group1,group2,meandiff,lower,upper,reject
Fair,Good,-429.8933,-740.4567,-119.3299,True
Fair,Ideal,-901.2158,-1180.5785,-621.8531,True
Fair,Premium,225.4999,-59.2739,510.2738,False
Fair,Very Good,-376.9979,-663.8694,-90.1263,True
Good,Ideal,-471.3225,-642.367,-300.2779,True
Good,Premium,655.3933,475.6466,835.1399,True
Good,Very Good,52.8954,-130.1565,235.9474,False
Ideal,Premium,1126.7157,1008.8058,1244.6257,True
Ideal,Very Good,524.2179,401.3281,647.1078,True
Premium,Very Good,-602.4978,-737.2366,-467.7591,True


In [25]:
# 회귀모델
import statsmodels.formula.api as smf

df = pd.read_csv("linear_regression_01.csv")
df.head()
model = smf.ols("y ~ x", data=df)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.089
Method:                 Least Squares   F-statistic:                     3.257
Date:                Thu, 22 Aug 2019   Prob (F-statistic):             0.0848
Time:                        14:43:13   Log-Likelihood:                -53.868
No. Observations:                  24   AIC:                             111.7
Df Residuals:                      22   BIC:                             114.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9909      0.908      1.091      0.2

In [26]:
np.exp(result.params)

Intercept    2.693690
x            1.452429
dtype: float64